In [174]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Hide Code"></form>''')

In [175]:
###Brand Pulse Dashboard
#Import Necessary Packages
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from sklearn.linear_model import LinearRegression

In [176]:
#Loading Data
PastYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/PastYearDataNew.csv", encoding='utf-8')	
CurrentYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/CurrentYearDataNew.csv", encoding='utf-8')	

#Combine Past and recent years sales data
SD = pd.concat([PastYearData, CurrentYearData], ignore_index=True)


In [177]:
#Convert Order_Date string to date and extract relevant date values
SD['Order_Date'] = pd.to_datetime(SD['Order_Date'], format= "%d-%b-%y")
SD['OD_Year'] = SD['Order_Date'].dt.strftime('%Y')
SD['OD_MonthNum'] = SD['Order_Date'].dt.strftime('%m')
SD['OD_MonthLab'] = SD['Order_Date'].dt.strftime('%B')
SD['OD_MonthDay'] = SD['Order_Date'].dt.strftime('%d')
SD['OD_WeekDay'] = SD['Order_Date'].dt.strftime('%A')


In [178]:
#Filter out any blank orderdate values
FilterSD =    SD[(SD['Order_Date']!="")
           #& (SD['Supplier']=="")
           ]

#Setup Interactive Global Filter
###Reconciliation_Status <-Set up check box filter
###Cost_Recording_Status <-Set up check box filter
###Order_Bucket>"Complete Bucket" <-Set up check box filter
###Order_Bucket>"Pending Bucket" <-Set up check box filter

In [179]:
#Group by Attribute set & year sold, then count number of sales
GroupedSD = (FilterSD.groupby(["attribute_set","OD_Year"], as_index=False)
                ['Order_Date'].
                agg({"count":"count"}).sort_values(["attribute_set","OD_Year"] , ascending = True))


#Identify all unique brands
AllBrands = GroupedSD.attribute_set.unique()

In [180]:
#Compiled Output functions
#Plot Brands sales per year 
def BrandPlotter(SB):
    BrandFilterSD = GroupedSD[(GroupedSD['attribute_set']==SB)]
    Year = np.array(BrandFilterSD["OD_Year"]).reshape((-1, 1))
    Sales = np.array(BrandFilterSD["count"])    
    
    MarginData = (FilterSD[(FilterSD['attribute_set']==SB)]
                 .groupby(["attribute_set","OD_Year"], as_index=False)
                 .agg({"Net_Retail_Price_CAD_":["sum"], 
                       "Total_Cost_CAD_":["sum"]
                      }))
    MarginData.columns = ["".join(x) for x in MarginData.columns.ravel()]
    
    MarginData = MarginData.assign(TotalMargin = lambda x: 
                 MarginData["Net_Retail_Price_CAD_sum"] - MarginData["Total_Cost_CAD_sum"])

    MarginData = MarginData.assign(PercentMargin = lambda x: 
                 (MarginData["TotalMargin"] / MarginData["Net_Retail_Price_CAD_sum"])*100)

    fig = plt.figure()
    ax1 = fig.add_axes([0.1, 0.5, 1, 1.5],
                   xticklabels=[], ylim=(Sales.min(), Sales.max()))
    ax2 = fig.add_axes([0.1, 0.1, 1, .6],
                   ylim=(MarginData["PercentMargin"].min(), MarginData["PercentMargin"].max()))

    ax1.plot(BrandFilterSD["OD_Year"],BrandFilterSD["count"])
    ax2.plot(MarginData["OD_Year"],MarginData["PercentMargin"])
    plt.show()

    LinearModel = LinearRegression().fit(Year,Sales)
    Slope = LinearModel.coef_
    RSqr= LinearModel.score(Year,Sales)
    
    if Slope > 0: 
        SlopeDirection = "Positive"
    else: 
        SlopeDirection = "Negative"
    
    print("The Slope of "+ SB+ " is "+SlopeDirection)

    
#TroubleShoot
if False:
    SB = "B&B Exhaust"
    BrandFilterSD = GroupedSD[(GroupedSD['attribute_set']==SB)]
    Year  = np.array(BrandFilterSD["OD_Year"]).reshape((-1, 1))
    Sales = np.array(BrandFilterSD["count"])    
    
#Need to make exception that if little data, error produced or just the tabular data outputed
    MarginData = (FilterSD[(FilterSD['attribute_set']==SB)]
                 .groupby(["attribute_set","OD_Year"], as_index=False)
                 .agg({"Net_Retail_Price_CAD_":["sum"], 
                       "Total_Cost_CAD_":["sum"]
                      }))
    MarginData.columns = ["".join(x) for x in MarginData.columns.ravel()]
    
    MarginData = MarginData.assign(TotalMargin = lambda x: 
                 MarginData["Net_Retail_Price_CAD_sum"] - MarginData["Total_Cost_CAD_sum"])

    MarginData = MarginData.assign(PercentMargin = lambda x: 
                 (MarginData["TotalMargin"] / MarginData["Net_Retail_Price_CAD_sum"])*100) 
    
    plt.plot(BrandFilterSD["OD_Year"],BrandFilterSD["count"])
    plt.plot(MarginData["OD_Year"],MarginData["PercentMargin"])
    plt.show()

    LinearModel = LinearRegression().fit(Year,Sales)
    Slope = LinearModel.coef_
    RSqr= LinearModel.score(Year,Sales)
    
    if Slope > 0: 
        SlopeDirection = "Positive"
    else: 
        SlopeDirection = "Negative"
    
    print("The Slope of "+ SB+ " is "+SlopeDirection)
    
    
###Net_Retail_Price_CAD_
###Total_Cost_CAD_
    
    #BrandPlotter("B&B Exhaust")

In [181]:
#Create Input Widgets
SelectedBrand =  widgets.Dropdown(options=AllBrands,description='Brands:',disabled=False)


In [184]:
%matplotlib inline
#interact(BrandPlotter, SB = SelectedBrand)